<a href="https://colab.research.google.com/github/JARAM2021/NLP_Study/blob/nykim/nykim/week5/week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataLoader
- PyTorch의 Dataset과 DataLoader를 이용하여 방대한 데이터를 미니배치 단위로 처리
- 데이터를 랜덤하게 섞어 학습의 효율성을 향상
- 여러 개의 GPU를 사용해 병렬 처리로 학습 가능

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
cd /content/gdrive/MyDrive/Colab Notebooks/2021_NLP_STUDY/

/content/gdrive/MyDrive/Colab Notebooks/2021_NLP_STUDY


## Kobert Download

In [ ]:
!pip install mxnet-cu101mkl
!pip install gluonnlp pandas tqdm
!pip install sentencepiece

     |████████████████████████████████| 712.3MB 23kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 8.8MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595684 sha256=219cba0ca23b0ee45e02b359e4bcbaef0931be3bf582851e882e4886573e281b
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 9.8MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-74cxtmzx
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-74cxtmzx
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=dba274d3160bc325179e033a0d761b86b159a6db254062b07bb3d5d93da552b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-69fe7phq/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
device = torch.device("cuda:0")

## Tokenizer

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.6MB/s 
     |████████████████████████████████| 901kB 33.6MB/s 
     |████████████████████████████████| 3.3MB 39.3MB/s 


In [ ]:
# from gluonnlp.data import SentencepieceTokenizer
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

bertmodel, vocab = get_pytorch_kobert_model()

tok_path = get_tokenizer()
tokenizer = nlp.data.BERTSPTokenizer(tok_path, vocab, lower=False)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
using cached model


In [ ]:
print(tokenizer('한국어 모델을 공유합니다.'))

['▁한국', '어', '▁모델', '을', '▁공유', '합니다', '▁', '.']


## Train/Test data splitter

In [ ]:
# from sklearn.model_selection import train_test_split
# from torch.utils.data import Subset
import pandas as pd

def train_test_splitter(csv_file) :
    dataset = pd.read_csv(csv_file)
    # train_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=split)
    # print(len(train_idx))
    # print(len(test_idx))
    # train_set = Subset(dataset, train_idx)
    # test_set = Subset(dataset, test_idx)
    return dataset

In [ ]:
dataset = train_test_splitter('./week5_data/ko_train_label.csv')

In [ ]:
dataset

,id,document,toxic,obscene,threat,insult,identity_hate
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,1,0,0,0,0
1,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,1,0,0,0,0
2,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,1,0,0,0,0
3,7797314,원작의 긴장감을 제대로 살려내지못했다.,1,0,0,0,0
4,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,1,0,0,0,1
...,...,...,...,...,...,...,...
9994,7448293,혹시나 그래도 카메론디아즈니까 하고봤는데...먼이런영화를..이도저도아닌 .아암튼 시...,1,0,0,0,0
9995,5824024,10점주는것들 한국영화는 1점주네. M창,1,0,0,0,0
9996,6420437,영상도 아름답고 뭘 말하는지 알겠지만 그렇기 때문에 짜증나고 답답하다,1,0,0,0,0
9997,6777278,영화를 왜 영화라고 하는지 모르는 애들이 나왔네.,1,0,0,0,0


## Dataset / DataLoader

In [ ]:
!pip install soynlp

     |████████████████████████████████| 419kB 8.1MB/s 


In [ ]:
from soynlp.normalizer import *

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import gluonnlp as nlp
import numpy as np
from torch import nn
class HateSpeechDataset(Dataset) :
    def __init__(self, csv_file, bert_tokenizer, max_len, pad, pair) :
        self.transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.hatespeech = pd.read_csv(csv_file)
    
    def __len__(self) :
        return len(self.hatespeech)
    
    def __getitem__(self, idx) :
        instance = self.hatespeech.iloc[idx]
        sentence = transform(repeat_normalize(instance[1]))
        label = instance[2]
        
        return {'sentence' : np.array(sentence, dtype=string_),
                'label' : label}
# class HateSpeechDataset(Dataset) :
#     def __init__(self, dataset, bert_tokenizer, sent_idx, toxic_idx, obscene_idx, threat_idx, insult_idx, identity_hate_idx, max_len, pad, pair) :
#         transform = nlp.data.BERTSentenceTransform(
#             bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
#         self.sentences = [transform([repeat_normalize(i[sent_idx])]) for i in dataset]
#         self.toxic = [np.int32(i[toxic_idx]) for i in dataset]
#         self.obscene = [np.int32(i[obscene_idx]) for i in dataset]
#         self.threat = [np.int32(i[threat_idx]) for i in dataset]
#         self.insult = [np.int32(i[insult_idx]) for i in dataset]
#         self.identity_hate = [np.int32(i[identity_hate_idx]) for i in dataset]
    
#     def __getitem__(self, i) :
#         return (self.sentences[i] + (self.labels[i], ))
    
#     def __len__(self) :
#         return (len(self.labels))


In [ ]:
dataset.iloc[1][1]

'교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'

In [ ]:
# train_data = HateSpeechDataset(dataset, tokenizer, 1, 2, 3, 4, 5, 6, 512, True, False)
train_data = HateSpeechDataset('./week5_data/hate_speech_binary_dataset.csv', tokenizer, 128, True, False)

In [ ]:
train_data